In [ ]:
!pip install langchain langchain_openai langsmith langchain_community gradio



**Open .env file in this folder and observe that we have configured OPENAI_API_KEY. Replace it with your own key or key given by me**

The Code in the below cell will load the .env file and set environment variables.

**Write the code in the below cell and execute it**

In [ ]:
from dotenv import load_dotenv

load_dotenv()


**Let us see if the model remembers previous messages**

If you execute the below code, you will understand that the model will not remember our previous messages by default

In [ ]:
from langchain_openai import  ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

model = ChatOpenAI()
firstresponse = model.invoke([HumanMessage(content="Hi! I'm Siva")])

print(firstresponse.content)
secondresponse = model.invoke([HumanMessage(content="What's my name?")])

print(secondresponse.content)


**Now Let us Pass all the messages manually to the model and observe**

In [ ]:
thirdresponse = model.invoke(
    [
        HumanMessage(content="Hi! I'm Siva"),
        AIMessage(content="Hello Siva! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

print(thirdresponse.content)

**Now let us add chatmessagehistory manually**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | model
chain

**Now, Let us create ChatMessageHistory , add messages and responses manually**

In [ ]:
from langchain.memory import ChatMessageHistory


chat_message_history = ChatMessageHistory()

chat_message_history.add_user_message(
    "Translate this sentence from English to French: I love programming."
)
response = chain.invoke(
    {"messages": chat_message_history.messages}
)
response


**Now, let us add the response from AI into chat message history**

**Then add our new usermessage also into chat message history**

**If you invoke the chain with all messages in chat message history, you will get response which makes us feel like LLM has memory**

**Understand and execute below code**

In [ ]:
chat_message_history.add_ai_message(AIMessage(response.content))
#chat_message_history.messages

chat_message_history.add_user_message("What did i ask u?")

finalresponse =  chain.invoke({ "messages": chat_message_history.messages })

chat_message_history.add_ai_message(AIMessage(finalresponse.content))
(chat_message_history.messages)

**Now let us use RunnableWithMessageHistory. Let us see how it maintains message history automatically**

**Firstly, let us define a function which returns an instance of ChatMessageHistory based on session_id which is default configurable parameter**

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

store = {}


def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

**Observe how we are passing get_session_history to RunnableWithMessageHistory Internally, RunnableWithMessageHistory will use get_session_history to get BaseChatMessageHistory for current session**



---





Understand the below code and execute it in a cell.
The below code goes through loop for 2 times asking you to enter your message.

For the first time, enter the message "My name is Siva"
Second time enter the message "What is my name?"

Once u get response, you will understand that how memory works


In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
count =0
config = {"configurable": {"session_id": "1122"}}

model_with_message_history = RunnableWithMessageHistory(model, get_session_history)

while(count < 2):
    content = input("Enter your message >> ")
    result = model_with_message_history.invoke(
        [HumanMessage(content=content)],
        config=config,
    )
    count = count + 1
    print(result.content)


model_with_message_history.get_session_history("1122").messages

# Let us build a chat bot which has memory

**Understand the below code and execute it**

In [ ]:
import gradio as gr
from langchain_core.runnables.history import RunnableWithMessageHistory

#session_id = input("Enter a session id >> ")

def predict(message, history):
   config = {"configurable": {"session_id": "1122"}}
   print(config)
   model_with_message_history = RunnableWithMessageHistory(model, get_session_history)

   response1= model_with_message_history.invoke(
        [HumanMessage(content=message)],
        config=config,
    ).content
   print(response1)

   print(model_with_message_history.get_session_history("1122").messages)
   return response1



gr.ChatInterface(predict).launch(debug=True)

**Now Let us use ChatPromptTemplate with RunnableWithMessageHistory**

Define a prompt using ChatPromptTemplate and then create a chain using it  as shown below



In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [  (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability .",
        ),  MessagesPlaceholder(variable_name="mychat_history"),
        ("human" ,"{input}")
    ]
)

chain =  prompt | model
chain

**Observe how we have Created  RunnableWithMessageHistory using the above chain. Also Observe input_messages_key and  history_messages_key**

In [ ]:
model_with_message_history = RunnableWithMessageHistory(
    chain, get_session_history, input_messages_key="input", history_messages_key="mychat_history"
)
model_with_message_history


**Understand how we are creating a chatbot using the below code and execute it in a cell**

In [ ]:
def predict(message, history):
   config = {"configurable": {"session_id": "1298"}}
   print(config)


   response1= model_with_message_history.invoke(
        {"input": message},
        config=config,
    ).content
   print(response1)

   print(model_with_message_history.get_session_history("1298").messages)
   return response1



gr.ChatInterface(predict).launch(debug=True)

**Let us see how to stream the result**

**Execute the below code in   cell and understand it**

In [ ]:
content = input(">> ")

streamtresult = model_with_message_history.stream(
       {"input": content},
        config=config,
    )
for token in streamtresult:
  print(token.content,end='|')

**Let is understand how to customize**

get_session_history functioned defined below uses a combination of user_id and conversation_id

Observe how we are customizing  RunnableWithMessageHistory using ConfigurableFieldSpec

**Execute the below cell and understand it**


In [ ]:
from langchain_core.runnables import ConfigurableFieldSpec

store = {}


def get_session_history(user_id: str, conversation_id: str) -> ChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]


chain = prompt | model
model_with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input", history_messages_key="mychat_history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id", annotation=str, 
        ),
        ConfigurableFieldSpec(
            id="conversation_id", annotation=str, 
        ),
    ],
)

config = {"configurable": {"user_id": "1", "conversation_id": "1"}}
count =0
while(count < 2):
    content = input(">> ")
    result = model_with_message_history.invoke(
       {"input": content,"language" : "Hindi"},
        config=config,
    )
    count = count + 1

    print(result.content)
model_with_message_history.get_session_history("123","1").messages

**Using FileChatMessageHistory to persist messages**

**Execute the below code to understand how chatmessage history is saved to a file**

In [ ]:
from langchain_community.chat_message_histories import FileChatMessageHistory


store = {}


def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = FileChatMessageHistory(user_id++conversation_id++"savedconversationmessages.json")
    return store[(user_id, conversation_id)]